# Business Analytics: Customer Segmentation on Online Retail Transaction Data
**Data Source: https://www.kaggle.com/mathchi/online-retail-ii-data-set-from-ml-repository**

**Data Analytics Created by Tableau can be viewed at:**

https://public.tableau.com/profile/minli.yu4995#!/vizhome/UK_MY/SalesAnalysisrofaUKOn-lineStore

**Note:** 

An Online Retail data set contains all the transactions occurring for a UK-based and registered, non-store online sales between 01/12/2009 and 09/12/2011. The company mainly sells unique all-occasion gift-ware. Many customers of the company are wholesalers.**

**Attribute Information provided by the dataset:** 

* InvoiceNo: Invoice number. Nominal. A 6-digit integral number uniquely assigned to each transaction. If this code starts with the letter 'c', it indicates a cancellation.
* StockCode: Product (item) code. Nominal. A 5-digit integral number uniquely assigned to each distinct product.
* Description: Product (item) name. Nominal.
* Quantity: The quantities of each product (item) per transaction. Numeric.
* InvoiceDate: Invice date and time. Numeric. The day and time when a transaction was generated.
* UnitPrice: Unit price. Numeric. Product price per unit in sterling (Â£).
* CustomerID: Customer number. Nominal. A 5-digit integral number uniquely assigned to each customer.
* Country: Country name. Nominal. The name of the country where a customer resides.

## 1. System setup/Loading dataset

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# Load the dataset
df_2009_2010 = pd.read_excel("../input/online-retail-ii-data-set-from-ml-repository/online_retail_II.xlsx", sheet_name = "Year 2009-2010")
df_2010_2011 = pd.read_excel("../input/online-retail-ii-data-set-from-ml-repository/online_retail_II.xlsx", sheet_name = "Year 2010-2011")

In [ ]:
# Concatenate the data frames of two years
df_concat = pd.concat([df_2009_2010,df_2010_2011],axis=0,ignore_index=True)
# Copy the dataset for pre-precessing/analytics
df=df_concat.copy()

## 2. Pre-process the dataset

In [ ]:
# 1.Eyeball the data
df.head()

In [ ]:
df.info()

In [ ]:
# 2. Check if there is records with Null values
print(f"Fields with null values:\n{df.isnull().any()}")
# Null values in CustomerID: some orders may purchased by non-member-> no Customer No.?

In [ ]:
# 3. For the purpose of RFM analysis, drop the order without customer ID (this customers are not trackable)
df.drop(index = df[df['Customer ID'].isnull()].index, inplace = True)
print(f"Fields with null values:\n{df.isnull().any()}")

In [ ]:
# 4. Detact the abnormal records (Invoice start with 'C', but not a cancellation invocie, and vice verer)
print("Any invoice starts with 'C' but is not a cancelation order ?",((df[df['Invoice'].str.startswith("C",na=False)]['Quantity']) > 0).any())
print("Any invoice not starts with 'C' but is a cancelation order ?", ((df[df['Invoice'].str.startswith("C",na=False) == False]['Quantity']) < 0).any())

In [ ]:
# 5. Calculate the sales amount of each order / Indicated whether the order is cancelled (chargeback) / turn Customer ID into INT
df['Sales'] = df['Quantity']*df['Price']
df['Chargeback'] = df['Invoice'].str.startswith("C",na=False) # show False (na=False) if element tested is not start with C (not a string).
df['Customer ID'] = df['Customer ID'].astype(int)

In [ ]:
# 6. Present the cleaned table
print("Show cleaned data set")
df.head()
# df.info()

## 3a. Data Analytics

In [ ]:
import matplotlib.pyplot as plt
import datetime
import calendar
%matplotlib inline

In [ ]:
# Navigate the sales trend over months
df['Month']=df['InvoiceDate'].dt.month
df_plot=df.loc[:,['Sales','Month','Chargeback']].sort_values(by=['Month'])
# Prepare the data: Calculate the sales and chargeback amount
x=df_plot.Month.unique()
y1=df.groupby('Month')['Sales'].sum().values/1e6
y2=df[df.Chargeback==True].groupby('Month').Sales.sum().values/-1e6

In [ ]:
# Sales trend Figure:
#1 Figure name and size
f1=plt.figure(figsize=(16,8))
plt.gcf()
#2 Grid
plt.grid(alpha=0.4)
plt.axis=([0,10,0,10])
# plt.xlim(0.5,12.5)
plt.ylim(-.2,3)
# 3 coordinate,tick scale/tick label/label rotation/font size
plt.xticks(x,rotation = 0.1,fontsize=16)
plt.xlabel('Months',fontsize=16)
plt.yticks(fontsize=16)
plt.ylabel(u'Total Sales (Minion \u00a3)',fontsize=16)
#4 plot y axis(#0022FF rgb(0,255))
plt.plot(x,y1,label='Sales Amount',color="blue",marker='o',markersize=10)
plt.plot(x,y2,label='Chargeback',color="red",marker = 'x',markersize=10)
#5 Legend#
plt.legend(loc='upper left',fontsize=20)
#6 Corrdinate showpoints
for a,b in zip(x,y1):
    plt.annotate('%.2f'%(b),xy=(a,b),xytext=(-15,15),textcoords='offset points',fontsize=12)
for a,b in zip(x,y2):
    plt.annotate('(%.2f)'%(b),xy=(a,b),xytext=(-15,15),textcoords='offset points',fontsize=12)    
plt.show()
plt.clf()

* It showed that the sales amount started to increase from Auguest. 
* It reached a maximum at November and dropped down a little bit at November. 
* This make sense since all customers tended to order their stocks for Christmas sale proactively.
* It is not suprised that the sales amount dramatically declined in January.

## 3b. Data analytics: RFM Model

In [ ]:
# 1.Only consider the non-chargeback orders when establishing the RFM models
df_C=df[df['Chargeback']==False]

In [ ]:
# 2. An invoice is ordered at certain time, which can include several products -> Group the invoice and create a pivot table
table=df_C.pivot_table(index=['Invoice'],values=['InvoiceDate','Customer ID','Sales'],aggfunc={'InvoiceDate':'first','Customer ID':'first','Sales':'sum'})
table.head()

In [ ]:
# 3. Group the customer ID to form a RFM table
table.rename(columns={'InvoiceDate':'R','Sales':'M'},inplace=True)
table['F'] = 1
table_RMF=table.pivot_table(index=['Customer ID'],values=['R','M','F'],aggfunc={'R':'max','M':'sum','F':'count'})
# 4. Normalised the recency, using the most recent day in the data set as the reference day
table_RMF['R']=(table_RMF['R'].max()-table_RMF['R']).dt.days
table_RMF.head()
# 5. Ignore the one off customers
table_RMF.drop(index=table_RMF[table_RMF['F']==1].index, inplace = True)

In [ ]:
# 5. Simple Statistic analysis on the RFM table
table_RMF.describe()
# 4314 customers: F: Frequency (no. of orders), M: Monetary (UK Pound), Recency (days)

In [ ]:
# End. Write the table into excel files for further analytics in Tableau
# df.to_csv('customer_segmentation.csv', index = True)
table_RMF.to_csv('RMF_table.csv',index = True)

## 4. Established a k-means clustering model

class sklearn.cluster.KMeans(n_clusters=8, init='k-means++', n_init=10, max_iter=300, tol=0.0001, precompute_distances='auto', verbose=0, random_state=None, copy_x=True, n_jobs=None, algorithm='auto')

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans

In [ ]:
#1 Standardise the data in RFM table
np.random.seed(42)
df_std=StandardScaler()
df_std=df_std.fit_transform(table_RMF)
x=df_std[:,0]
y=df_std[:,1]
z=df_std[:,2]

In [ ]:
#2 K-means clustering
km=KMeans(n_clusters=6)
res=km.fit(df_std)
clusterNo=res.predict(df_std)

In [ ]:
#3 Show the number of customers in each segment
# table_RMF['clusterNo']=clusterNo
seg,segNo=np.unique(clusterNo,return_counts=True)
print(seg,'\t',segNo) 

In [ ]:
# Plot the 3D clustering results
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
f1=plt.figure(figsize=(16,8))
ax1 = f1.add_subplot(111, projection='3d')
ax1.set_xlabel('Recency',size=16)
ax1.set_ylabel('Frequency',size=16)
ax1.set_zlabel('Monetary',size=16)
ax1.scatter(x,y,z,c=clusterNo,cmap='plasma')

# Summary of Customer Segmentation

* Seg 1 : Navy: Low Recency, Low ~ Medium Frequency, High Monetary ->  Best Customers (Monetatry is the dominant factor)
* Seg 2 : Orange: Low Recency, Low Frequency, Medimum Monetary -> High-spending Active Loyal Customers 
* Seg 3 : Purple: Low to Medium Recency, Low Frequency, Low Monetary -> Low-Spending Active Loyal Customers
* Seg 4 : Red: Medium, Low ~ Medium Frequency, Low to Medium Monetary -> Churning Customers
* Seg 5 : Yellow: High recency, Medium Frequency, low Monetary ->retail customers
* Seg 6 : Pink: High recency,High Frequency, Low Monetary  ->retail customers
* Note: Segments 1 to 4 are wholesaler while 5 to 6 are very likey retail customers (they ordered frequently with small amount)  
